In [29]:
import pandas as pd
import numpy as np

from datetime import datetime
from dateutil.relativedelta import relativedelta
from scipy.stats import loguniform
from sklearn.model_selection import PredefinedSplit, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error

In [30]:
df = pd.read_csv('data.csv', parse_dates=[1]).sort_values(by = "date")
df['date'] = df['date'].dt.to_period('M')
df = df[277:].reset_index(drop=True)

In [31]:
df.head()

,optid,date,secid,cp_flag,strike,bid,ask,volume,openint,impvol,...,vix,dhedged_return_mid,dhedged_return_spot,dhedged_return_spot_gamma,dhedged_return_mid_delev,dhedged_return_spot_delev,dhedged_return_spot_gamma_delev,IV_mness_deriv_1,IV_ttm_deriv_1,short_rate
0,10758593,1996-03,108105,C,630.0,20.000,20.500,7.0,2885.0,0.163628,...,16.54,-0.162051,-0.005116,-0.453583,-0.007654,-0.000242,-0.021423,-0.034178,0.000045,0.054624
1,10093854,1996-03,108105,P,650.0,17.500,18.250,126.0,2383.0,0.143119,...,16.54,-0.161114,-0.004490,-0.430225,-0.008292,-0.000231,-0.022141,-0.025132,0.000005,0.054624
2,10453184,1996-03,108105,P,675.0,33.625,34.625,0.0,202.0,0.133780,...,16.54,-0.032381,-0.001723,-0.212260,-0.002190,-0.000117,-0.014357,-0.008622,-0.000010,0.054624
3,11075738,1996-03,108105,P,650.0,35.000,36.500,13.0,2376.0,0.153038,...,16.54,-0.034062,-0.001898,-0.684611,-0.005364,-0.000299,-0.107810,-0.029598,0.000015,0.054624
4,11727207,1996-03,108105,C,655.0,30.250,31.250,7120.0,8000.0,0.146335,...,16.54,-0.019832,-0.000951,-0.160711,-0.001621,-0.000078,-0.013133,NaN,NaN,0.054624


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180537 entries, 0 to 180536
Data columns (total 37 columns):
 #   Column                           Non-Null Count   Dtype    
---  ------                           --------------   -----    
 0   optid                            180537 non-null  int64    
 1   date                             180537 non-null  period[M]
 2   secid                            180537 non-null  int64    
 3   cp_flag                          180537 non-null  object   
 4   strike                           180537 non-null  float64  
 5   bid                              180537 non-null  float64  
 6   ask                              180537 non-null  float64  
 7   volume                           180537 non-null  float64  
 8   openint                          180537 non-null  float64  
 9   impvol                           180537 non-null  float64  
 10  delta                            180537 non-null  float64  
 11  gamma                            180537

In [33]:
def get_summary(df_col):
    print(f'\nSummary of {df_col.name}:')
    print(f'Mean: {np.round( df_col.mean(),2)}')
    print(f'Median: {np.round( df_col.median(),2)}')
    print(f'Standard deviation: {np.round( df_col.std(),2)}')
    print(f'Number of observations: {np.round( df_col.count(),2)}')    

In [34]:
def add_months(date ,period_to_add=1):
    return (datetime.strptime(date, '%Y-%m') + relativedelta(months=period_to_add)).strftime('%Y-%m')

In [35]:
features = [ 'midprice', 'mness', 'ttm', 'embed_lev', 'impvol', 'gamma', 'vega', 'theta']

In [36]:
print('\nSummary statistics for Call options\n')
df[df["cp_flag"] == 'C'][features].apply(lambda x: get_summary(x), axis=0);


Summary statistics for Call options


Summary of midprice:
Mean: 91.46
Median: 37.39
Standard deviation: 163.66
Number of observations: 69044

Summary of mness:
Mean: 0.49
Median: 0.31
Standard deviation: 1.3
Number of observations: 69044

Summary of ttm:
Mean: 208.87
Median: 91.0
Standard deviation: 224.3
Number of observations: 69044

Summary of embed_lev:
Mean: 22.17
Median: 14.24
Standard deviation: 21.85
Number of observations: 69044

Summary of impvol:
Mean: 0.21
Median: 0.18
Standard deviation: 0.15
Number of observations: 69044

Summary of gamma:
Mean: 0.0
Median: 0.0
Standard deviation: 0.0
Number of observations: 69044

Summary of vega:
Mean: 240.76
Median: 176.03
Standard deviation: 230.11
Number of observations: 69044

Summary of theta:
Mean: -56.13
Median: -43.84
Standard deviation: 51.24
Number of observations: 69044


In [37]:
print('\nSummary statistics for Put options\n')

df[df["cp_flag"] == 'P'][features].apply(lambda x: get_summary(x), axis=0);


Summary statistics for Put options


Summary of midprice:
Mean: 71.67
Median: 14.25
Standard deviation: 154.16
Number of observations: 111493

Summary of mness:
Mean: -0.9
Median: -1.03
Standard deviation: 1.43
Number of observations: 111493

Summary of ttm:
Mean: 202.65
Median: 91.0
Standard deviation: 222.21
Number of observations: 111493

Summary of embed_lev:
Mean: 14.65
Median: 11.78
Standard deviation: 10.6
Number of observations: 111493

Summary of impvol:
Mean: 0.3
Median: 0.26
Standard deviation: 0.18
Number of observations: 111493

Summary of gamma:
Mean: 0.0
Median: 0.0
Standard deviation: 0.0
Number of observations: 111493

Summary of vega:
Mean: 178.92
Median: 105.07
Standard deviation: 205.27
Number of observations: 111493

Summary of theta:
Mean: -49.22
Median: -36.21
Standard deviation: 49.53
Number of observations: 111493


In [38]:
features += ['strike', 'spot_close', 'divrate', 'vix', 'short_rate', 'delta', 'date'] #+ ['optid']
train_features = features[1:-1]

In [39]:
clear_df = df[features]

In [40]:
start_date = '2007-01'

y_prices = clear_df[clear_df['date'] > start_date]['midprice']

In [41]:
def get_splits(df, num=False):
    train_pc = 0.85

    df_len = df.shape[0]
    separator = int(np.floor(df_len*train_pc))
    rest = df_len - separator

    if num:
        return df[:separator], df[separator:]
    else:
        return np.append(np.ones(separator)*-1,np.zeros(rest))

## OLS Regression

In [42]:
ols_predictions = np.array([])
ols_r_2 = np.array([])

In [ ]:
# OLS
for current_date in df[df['date'] > start_date]['date'].unique():
    
    x_train = clear_df[clear_df['date'] < current_date][train_features]
    y_train = clear_df[clear_df['date'] < current_date]['midprice']

    x_test = clear_df[clear_df['date'] == current_date][train_features]
    y_test = clear_df[clear_df['date'] == current_date]['midprice']
    
    ols = LinearRegression()
    ols.fit(x_train, y_train)
    
    y_pred = ols.predict(x_test)
    
    # Write predictions to array
    ols_predictions = np.append(ols_predictions, y_pred)
    
    # Write R^2 to array
    ols_r_2 = np.append(ols_r_2, ols.score(x_train, y_train))

In [ ]:
np.round(np.mean(ols_r_2), 3)

In [ ]:
# MSE
np.round(mean_squared_error(y_prices, ols_predictions), 3)

## Penalized Linear Regressions

### Ridge

In [28]:
temp = clear_df[clear_df['date'] < '2007-02']

In [156]:
temp

,midprice,mness,ttm,embed_lev,impvol,gamma,vega,theta,strike,spot_close,divrate,vix,short_rate,delta,date
0,20.250,-0.354854,35,21.172955,0.163628,0.011279,70.73290,-73.637880,630.0,641.43,0.026814,16.54,0.054624,0.668432,1996-03
1,17.875,0.223216,63,19.430897,0.143119,0.010436,104.38430,-32.682600,650.0,641.43,0.026814,16.54,0.054624,-0.541489,1996-03
2,34.125,0.917831,63,14.784788,0.133780,0.008116,75.88284,-13.032170,675.0,641.43,0.026814,16.54,0.054624,-0.786572,1996-03
3,35.750,0.065291,644,6.350139,0.153038,0.002773,307.54810,-3.496969,650.0,641.43,0.026814,16.54,0.054624,-0.353924,1996-03
4,30.750,-0.013444,189,12.236780,0.146335,0.005916,172.42900,-36.849540,655.0,641.43,0.026814,16.54,0.054624,0.573704,1996-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47621,5.850,1.971447,700,16.587442,0.103961,0.000631,257.18450,-9.800429,1900.0,1430.50,0.020623,10.40,0.053642,0.067834,2007-01
47622,19.900,-0.617211,245,13.744014,0.151504,0.001529,316.97500,-25.533650,1325.0,1430.50,0.020623,10.40,0.053642,-0.191196,2007-01
47623,9.200,-0.602407,56,35.585709,0.121717,0.004474,167.92540,-56.150670,1390.0,1430.50,0.020623,10.40,0.053642,-0.228863,2007-01
47624,41.600,0.801262,518,11.532409,0.117313,0.001793,609.83650,-38.818240,1600.0,1430.50,0.020623,10.40,0.053642,0.335371,2007-01


In [170]:
ps = PredefinedSplit(get_splits(temp))


ridge = Ridge()

space = dict()
space['solver'] = ['svd', 'cholesky', 'lsqr', 'sag']
space['alpha'] = np.array([1,0.1,0.01,0.001,0.0001,0])
space['fit_intercept'] = [True, False]
space['normalize'] = [True, False]

ridge_cv = GridSearchCV(ridge, param_grid=space, cv = list(PredefinedSplit(get_splits(temp)).split()), refit = False)

In [181]:
train, validation = get_splits(clear_df[clear_df['date'] < '2007-02'], num = True)

x_train = train[train_features]
y_train = train['midprice']

x_valid = validation[train_features]
y_valid = validation['midprice']

In [185]:
temp

,midprice,mness,ttm,embed_lev,impvol,gamma,vega,theta,strike,spot_close,divrate,vix,short_rate,delta,date
0,20.250,-0.354854,35,21.172955,0.163628,0.011279,70.73290,-73.637880,630.0,641.43,0.026814,16.54,0.054624,0.668432,1996-03
1,17.875,0.223216,63,19.430897,0.143119,0.010436,104.38430,-32.682600,650.0,641.43,0.026814,16.54,0.054624,-0.541489,1996-03
2,34.125,0.917831,63,14.784788,0.133780,0.008116,75.88284,-13.032170,675.0,641.43,0.026814,16.54,0.054624,-0.786572,1996-03
3,35.750,0.065291,644,6.350139,0.153038,0.002773,307.54810,-3.496969,650.0,641.43,0.026814,16.54,0.054624,-0.353924,1996-03
4,30.750,-0.013444,189,12.236780,0.146335,0.005916,172.42900,-36.849540,655.0,641.43,0.026814,16.54,0.054624,0.573704,1996-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47621,5.850,1.971447,700,16.587442,0.103961,0.000631,257.18450,-9.800429,1900.0,1430.50,0.020623,10.40,0.053642,0.067834,2007-01
47622,19.900,-0.617211,245,13.744014,0.151504,0.001529,316.97500,-25.533650,1325.0,1430.50,0.020623,10.40,0.053642,-0.191196,2007-01
47623,9.200,-0.602407,56,35.585709,0.121717,0.004474,167.92540,-56.150670,1390.0,1430.50,0.020623,10.40,0.053642,-0.228863,2007-01
47624,41.600,0.801262,518,11.532409,0.117313,0.001793,609.83650,-38.818240,1600.0,1430.50,0.020623,10.40,0.053642,0.335371,2007-01


In [186]:
ridge_cv.fit(temp[train_features], temp['midprice'])

GridSearchCV(cv=[(array([    0,     1,     2, ..., 40479, 40480, 40481]),
                  array([40482, 40483, 40484, ..., 47623, 47624, 47625]))],
             estimator=Ridge(),
             param_grid={'alpha': array([1.e+00, 1.e-01, 1.e-02, 1.e-03, 1.e-04, 0.e+00]),
                         'fit_intercept': [True, False],
                         'normalize': [True, False],
                         'solver': ['svd', 'cholesky', 'lsqr', 'sag']},
             refit=False)

In [169]:
list(PredefinedSplit(get_splits(temp)).split())

[(array([    0,     1,     2, ..., 40479, 40480, 40481]),
  array([40482, 40483, 40484, ..., 47623, 47624, 47625]))]

In [167]:
ridge_cv.best_params_

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [165]:
loguniform(1e-5, 100)

In [118]:
# Ridge
for current_date in df[df['date'] > start_date]['date'].unique():
    
    train, validation = get_splits(clear_df[clear_df['date'] < current_date])
    
    space = dict()
    space['solver'] = ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
    space['alpha'] = loguniform(1e-5, 100)
    space['fit_intercept'] = [True, False]
    space['normalize'] = [True, False]
    
    x_train = train[train_features]
    y_train = train['midprice']

    x_valid = validation[train_features]
    y_valid = validation['midprice']
    
    x_test = clear_df[clear_df['date'] == current_date][train_features]
    y_test = clear_df[clear_df['date'] == current_date]['midprice']